In [1]:
import threading
import time
import tkinter as tk

import cv2
import keyboard
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageTk

# Criar janela
janela = tk.Tk()
janela.title("Atualização com loop real (thread)")
janela.attributes('-fullscreen', True)

janela.update()
largura = janela.winfo_width()
altura = janela.winfo_height()

In [2]:
class inCeneObject:
    def __init__(self, father, posX, posY):
        self.posX = posX
        self.posY = posY
        self.father = father

    def getRealPosition(self) -> (int, int):
        if(self.father == None):
            return self.posX, self.posY
        else:
            Xf, Yf = self.father.getRealPosition()
            return self.posX + Xf, self.posY + Yf


class Hitbox(inCeneObject):
    def __init__(self, posX, posY, tags: list, father=None):
        super().__init__(father, posX, posY)
        self.tags = tags
        self.in_contact_HB_List = []
        self.father = father

class Point_Hitbox(Hitbox):
    def __init__(self, posX, posY, tags: list, father=None):
        super().__init__(father, posX, posY)

class Square_Hitbox(Hitbox):
    def __init__(self, posX, posY, altura, largura, tags: list, father=None):
        super().__init__(father, posX, posY)
        self.altura = altura
        self.largura = largura




class Entity(inCeneObject):
    def __init__(self, startposX, startposY, tags=[], father=None):
        super().__init__(father, startposX, startposY)

        self.tags = tags
        self.father = father

        self.sprites = []
        self.hitboxes = []


class Sprite(inCeneObject):
    def __init__(self, full_sprite, posX, posY, actual_sprite=0, count_time=0, sprite_size = 32, visible = True, father=None, tags=None):
        super().__init__(father, posX, posY)

        self.full_sprite = full_sprite
        self.img_actual = full_sprite
        self.actual_sprite = actual_sprite
        self.count_time = count_time
        self.sprite_size = sprite_size

        self.father = father
        self.visible = visible
        self.tags = tags


    def simple_animation(self, intent):    # intent element:
    #   [frames_count, sprite_Y, sprite_X, sprite_l, sprite_h]

        x = self.actual_sprite
        if(self.count_time == 0):
            a0, a1 = self.sprite_size*intent[x][1], self.sprite_size*(intent[x][3]+intent[x][1])
            b0, b1 = self.sprite_size*intent[x][2], self.sprite_size*(intent[x][4]+intent[x][2])
            print(a0, a1)
            print(b0, b1)
            self.img_actual = self.full_sprite[a0:a1, b0:b1]


        if(self.count_time >= intent[self.actual_sprite][0]):
            self.count_time = 0
            self.actual_sprite = self.actual_sprite + 1
        else:
            self.count_time = self.count_time + 1

        if(self.actual_sprite > len(intent)-1):
            self.actual_sprite = 0



class Camera:
    def __init__(self, posX, posY, father, altura, largura, fullscreen=True, crossHair=False, tags=[]):
        self.posX = posX
        self.posY = posY
        self.fullscreen = fullscreen

        self.altura = altura
        self.largura = largura
        if(crossHair == True):
            self.posY -= self.altura/2
            self.posX -= self.largura/2


        self.father = father
        self.tags = tags


In [3]:
Objects_List = []
Sprite_List = []
Camera_List = []
teclas_pressionadas = set()

Square_Hitbox_List = []
Point_Hitbox_List = []



prot = Entity(130, 100, ['PC'])

prot_hitbox = Point_Hitbox(0, 0, ['char'], prot)
prot.hitboxes.append(prot_hitbox)
Point_Hitbox_List.append(prot_hitbox)

prot_sprite = Sprite((plt.imread('imagem_com_plt.png') * 255).astype("uint8"), -50, -60, father=prot)
prot.sprites.append(prot_sprite)


block = Entity(160, 200, ['wall'])

block_hitbox = Square_Hitbox(10, 20, 200, 300, ['wall'], prot)
block.hitboxes.append(block_hitbox)
Square_Hitbox_List.append(block_hitbox)

block_sprite = Sprite((plt.imread('bloco.png') * 255).astype("uint8"), -50, -60, father=block)
block.sprites.append(prot_sprite)


camera_main = Camera(0, 0, prot, 500, 500)

In [4]:



def simple_game_loop():

    if 'w' in teclas_pressionadas:
        prot.posY -= 10
    if 's' in teclas_pressionadas:
        prot.posY += 10
    if 'a' in teclas_pressionadas:
        prot.posX -= 10
    if 'd' in teclas_pressionadas:
        prot.posX += 10



    tela = np.zeros((altura, largura, 4), dtype="uint8")

    posX, posY = prot_sprite.getRealPosition()
    prot_sprite.simple_animation([[5, 0, 0, 1, 1], [3, 1, 0, 1, 1], [7, 0, 0, 2, 1]])
    tmp = prot_sprite.img_actual
    shape = tmp.shape
    tela[posY:posY+shape[0], posX:posX+shape[1]] = tmp

    posX, posY = block_sprite.getRealPosition()
    tmp = block_sprite.img_actual
    tela[posY:posY+tmp.shape[0], posX:posX+tmp.shape[1]] = tmp

    return tela

In [5]:


img_array = np.zeros((altura, largura, 3), dtype=np.uint8)



def verificar_teclas():
    global teclas_pressionadas

    teclas_atuais = set()

    # Captura todas as teclas conhecidas e verifica quais estão pressionadas
    for tecla in keyboard.all_modifiers.union(set('abcdefghijklmnopqrstuvwxyz0123456789')):
        if keyboard.is_pressed(tecla):
            teclas_atuais.add(tecla)

    if teclas_atuais != teclas_pressionadas:
        teclas_pressionadas.clear()
        teclas_pressionadas.update(teclas_atuais)
        print("Teclas pressionadas:", sorted(teclas_pressionadas))

    janela.after(10, verificar_teclas)

def atualizar_imagem():

    imagem_rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    imagem_pil = Image.fromarray(imagem_rgb)
    imagem_tk = ImageTk.PhotoImage(imagem_pil)
    canvas.itemconfig(imagem_canvas, image=imagem_tk)
    canvas.image = imagem_tk  # Impede que a imagem seja coletada pelo garbage collector

def loop_em_thread():
    while True:
        img_array[0:len(img_array), 0:len(img_array[0])] = simple_game_loop()

        # Atualiza imagem na interface
        janela.after(0, atualizar_imagem)  # chama a atualização de forma segura no thread principal

        time.sleep(0.01)  # Espera 10ms


# Criar a imagem inicial
img_array = np.zeros((altura, largura, 4), dtype=np.uint8)


# Converter imagem inicial
imagem_rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
imagem_pil = Image.fromarray(imagem_rgb)
imagem_tk = ImageTk.PhotoImage(imagem_pil)

# Canvas
canvas = tk.Canvas(janela, width=largura, height=altura)
canvas.pack()
imagem_canvas = canvas.create_image(0, 0, anchor="nw", image=imagem_tk)




# Inicia o loop em uma thread separada
thread_loop = threading.Thread(target=loop_em_thread, daemon=True)
thread_loop.start()

thread_keys = threading.Thread(target=verificar_teclas, daemon=True)
thread_keys.start()

# Loop principal da janela
verificar_teclas()
janela.mainloop()


0 32
0 32
32 64
0 32
0 64
0 32
0 32
0 32
32 64
0 32
0 64
0 32
0 32
0 32
32 64
0 32
0 64
0 32
0 32
0 32
32 64
0 32
0 64
0 32
0 32
0 32
32 64
0 32
0 64
0 32
Teclas pressionadas: ['d']
0 32
0 32
Teclas pressionadas: ['d', 's']
32 64
0 32
Teclas pressionadas: ['d']
0 64
0 32
Teclas pressionadas: []
0 32
0 32
Teclas pressionadas: ['d']
32 64
0 32
Teclas pressionadas: ['d', 's']
0 64
0 32
Teclas pressionadas: ['d']
Teclas pressionadas: []
0 32
0 32
32 64
0 32
Teclas pressionadas: ['a']
0 64
0 32
0 32
0 32
Teclas pressionadas: []
32 64
0 32
0 64
0 32
0 32
0 32
Teclas pressionadas: ['d']
32 64
0 32
Teclas pressionadas: []
0 64
0 32
0 32
0 32
32 64
0 32
Teclas pressionadas: ['alt', 'left alt', 'right alt']
0 64
0 32
0 32
0 32
Teclas pressionadas: []
32 64
0 32
0 64
0 32
0 32
0 32
32 64
0 32
0 64
0 32
0 32
0 32
32 64
0 32
0 64
0 32
0 32
0 32
32 64
0 32
0 64
0 32
0 32
0 32
32 64
0 32
0 64
0 32
0 32
0 32
32 64
0 32
0 64
0 32
0 32
0 32
Teclas pressionadas: ['alt', 'left alt', 'right alt']
32 64
0 3